## Scraping Data from Website + Pandas

In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
# URL de la lista de animes por número de episodios
url = "https://en.wikipedia.org/wiki/List_of_anime_series_by_episode_count"

# Cabecera para evitar el bloqueo del servidor
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/125.0.0.0 Safari/537.36"
}

# Hacer la petición
page = requests.get(url, headers=headers)

# Crear el objeto BeautifulSoup
soup = BeautifulSoup(page.text, "html")

In [3]:
#print(soup)

In [4]:
#soup.find_all('table')[1]

In [5]:
#table = soup.find_all('table')[1]

In [6]:
table = soup.find('table', class_ ='wikitable sortable')

In [7]:
headers = [th.text.strip() for th in table.find_all('th')]

In [8]:
print(headers)

['No.', 'Series title', 'Started broadcasting', 'Finished broadcasting', 'Episode count', 'Runtime']


In [9]:
import pandas as pd

In [10]:
column_data=table.find_all('tr')

In [17]:
all_rows = []

last_rank = None  # último ranking válido

for row in column_data:
    for sup in row.find_all('sup'):
        sup.decompose()
    
    cells = [td.text.strip() for td in row.find_all('td')]
    
    if cells:
        # Suponiendo que la primera columna es el ranking
        if cells[0] == '' or not cells[0].isdigit():
            # Si está vacío o no es número, usar el último ranking válido
            cells[0] = last_rank
        else:
            # Actualizar el último ranking válido
            last_rank = cells[0]
        
        # Si la fila tiene menos columnas que los encabezados, rellenar repitiendo el último valor de cada columna
        while len(cells) < len(headers):
            cells.append(cells[-1])
        
        all_rows.append(cells)

#for r in all_rows:
    #print(r)

In [16]:
df = pd.DataFrame(all_rows, columns=headers)

df

,No.,Series title,Started broadcasting,Finished broadcasting,Episode count,Runtime
0,1,Sazae-san,"October 5, 1969",Currently in production,"2,771",20–26 minutes
1,2,Nintama Rantarō,"April 10, 1993",Currently in production,"2,504",10 minutes
2,3,Ojarumaru,"October 5, 1998",Currently in production,"2,127",10 minutes
3,4,Oyako Club,"October 3, 1994","March 30, 2013","1,818",5 minutes
4,5,Doraemon (1979),"April 2, 1979","March 18, 2005","1,787",11 minutes
...,...,...,...,...,...,...
173,118,"April 7, 2016","March 29, 2018",100,20–26 minutes,20–26 minutes
174,118,"October 4, 1997","September 25, 1999",100,20–26 minutes,20–26 minutes
175,118,"October 3, 2020","October 22, 2022",100,20–26 minutes,20–26 minutes
176,118,"April 1, 2007","March 29, 2009",100,20–26 minutes,20–26 minutes


In [19]:
df.to_csv(r'C:\Users\linkm\Documents\Python\Web_Scraping\Animes.csv',index = False)